# 라이브러리

In [ ]:
# 라이브러리
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set(style='whitegrid', font_scale=1.5)
import itertools
import warnings
warnings.filterwarnings('ignore')
import time

# 사이킷런

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, recall_score, precision_score, f1_score
from sklearn.preprocessing import LabelEncoder

## 모델
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

****데이터 가져오기****

In [ ]:
# 데이터프레임에 저장
train = pd.read_csv('../input/spaceship-titanic/train.csv')
test = pd.read_csv('../input/spaceship-titanic/test.csv')
sub_df =  test[['PassengerId']]
# 데이터 쉐이프 보기
print('트레인셋:', train.shape)
print('테스트셋:', test.shape)
train.head(20)

* PassengerId - 승객의 고유 ID. 그룹 4글자-그룹내번호 2글자 gggg_pp 로 이루어져있다. 같은 그룹이어도 가족이 아닐수 있다.(last name 기준)
* HomePlanet - 승객의 고향별. 
* CryoSleep - 냉동수면. 승객이 여정 중 생명활동을 멈췄는지 여부. 냉동수면에 든 승객은 선실안에만 있다.
* Cabin - 선실. 선실 넘버는 승객이 머무는 곳. 덱(아마 층)/숫자/사이드(deck/num/side)의 형태 side의 P(port)는 좌현, S(starboard)는 우현.
* Destination - 도착지. 승객이 내릴 별
* Age - 나이. 승객 나이
* VIP - 여정 중 스페셜 VIP 서비스에 돈을 지불했는지 여부.
* RoomService, FoodCourt, ShoppingMall, Spa, VRDeck - 룸서비스, 푸드코트, 쇼핑몰, 스파, VR덱 : 승객이 타이타닉호 우주선의 럭셔리 편의시설에 지불한 비용
* Name - 승객의 이름과 성
* Transported - 전송여부. 승객이 다른 차원으로 전송되었는지 여부. 이게 바로 예측해야 될 타겟

In [ ]:
sub_df.head()

In [ ]:
print(f'트레인셋 결측값:\n{train.isna().sum()}')
print('')
print(f'테스트셋 결측값:\n{test.isna().sum()}')

print(f'트레인셋 중복값: {train.duplicated().sum()}')
print(f'테스트셋 중복값: {test.duplicated().sum()}')

고유값, 데이터타입

In [ ]:
train.nunique()

In [ ]:
test.nunique()

In [ ]:
train.dtypes

numeric data로 변경해야 머신러닝 모델에 사용할 수 있음..

# EDA

타겟 분포

In [ ]:
# 그래프 사이즈
plt.figure(figsize=(6,6))

# 파이차트
train['Transported'].value_counts().plot.pie(explode=[0.1,0.1], autopct='%1.1f%%', shadow=True, textprops={'fontsize':16}).set_title("Target distribution")

연속되는 피쳐

In [ ]:
# 그래프 사이즈
plt.figure(figsize=(10,4))

# 히스토그램
sns.histplot(data=train, x='Age', hue='Transported', binwidth=1, kde=True)

# 히.꾸.(히스토그램 꾸미기).. 한국어좀 .. 
plt.title('나이분포')
plt.xlabel('나이 (년)')

* 0~5세 : 2배이상 전송됨 (5) child
* 6세~18세 : 유의미하게 전송됨 (4) adolescent
* 19세~25세 : 전송되지 않는 편 (1) adult
* 26세~40세 : 약간 전송되지 않는 편 (2) middleage
* 41세~ : 비슷한 확률 (3) old

** 편의시설(스파, 쇼핑몰, 푸드코트, VR덱),vip은 제외
** 도착지(Destination)과 VIP는 제외,

In [ ]:
train.head()

In [ ]:
amen_feats = ['VIP','RoomService','FoodCourt','ShoppingMall','Spa','VRDeck']

train.drop(amen_feats, axis=1, inplace=True)
test.drop(amen_feats, axis=1, inplace=True)


** 'HomePlanet'고향별의 경우 유로파(주로 전송됨), 지구(전송덜됨), 화성(비슷함) / 'CryoSleep냉동수면의 경우 냉동수면 상태가 true인경우 압도적으로 transported 됨.

In [ ]:
# 카테고리형 데이터 피쳐
cat_feats=['HomePlanet', 'CryoSleep']

# 그래프
fig=plt.figure(figsize=(10,16))
for i, var in enumerate(cat_feats):
    ax=fig.add_subplot(2,1,i+1)
    sns.countplot(data=train, x=var, axes=ax, hue='Transported')
    ax.set_title(var)
plt.show()

복합적인 데이터지표

In [ ]:
# 오브젝트 데이터 타입 형태 (글자들)
comp_feats=['PassengerId', 'Cabin' ,'Name']

# 확인
train[comp_feats].head()

승객아이디 - 앞에 숫자 4개는 그룹명, 뒤에 두개는 그룹내 사람수 
객실 - 덱 / 숫자 / 사이드
이름 - 이름 성

# Feature engineering

나이..(Age Grouping했는데 결국 안 씀 ..)

* 0~5세 : 2배이상 전송됨 (5) child
* 6세~18세 : 유의미하게 전송됨 (4) adolescent
* 19세~25세 : 전송되지 않는 편 (1) adult
* 26세~40세 : 약간 전송되지 않는 편 (2) middleage
* 41세~ : 비슷한 확률 (3) old

In [ ]:
# 트레인셋 새피쳐
train['Age_group']=np.nan
train.loc[train['Age']<=5,'Age_group']='Age_0-5'
train.loc[(train['Age']>5) & (train['Age']<=18),'Age_group']='Age_6-18'
train.loc[(train['Age']>18) & (train['Age']<=25),'Age_group']='Age_19-25'
train.loc[(train['Age']>25) & (train['Age']<=40),'Age_group']='Age_26-40'
train.loc[train['Age']>40,'Age_group']='Age_41+'

# 테스트셋 새피쳐
test['Age_group']=np.nan
test.loc[test['Age']<=5,'Age_group']='Age_0-5'
test.loc[(test['Age']>5) & (test['Age']<=18),'Age_group']='Age_6-18'
test.loc[(test['Age']>18) & (test['Age']<=25),'Age_group']='Age_19-25'
test.loc[(test['Age']>25) & (test['Age']<=40),'Age_group']='Age_26-40'
test.loc[test['Age']>40,'Age_group']='Age_41+'

# 그래프로 피쳐 분포 확인하기 
plt.figure(figsize=(10,4))
g=sns.countplot(data=train, x='Age_group', hue='Transported', order=['Age_0-5','Age_6-18','Age_19-25','Age_26-40','Age_41+'])
plt.title('Age group distribution')

승객그룹

In [ ]:
# 새 피쳐 - 그룹
train['Group'] = train['PassengerId'].apply(lambda x: x.split('_')[0]).astype(int)
test['Group'] = test['PassengerId'].apply(lambda x: x.split('_')[0]).astype(int)

# 새 피쳐 - 그룹사이즈
train['Group_size']=train['Group'].map(lambda x: pd.concat([train['Group'], test['Group']]).value_counts()[x])
test['Group_size']=test['Group'].map(lambda x: pd.concat([train['Group'], test['Group']]).value_counts()[x])

# 그래프로 피쳐 분포 확인하기 
plt.figure(figsize=(20,4))
plt.subplot(1,2,1)
sns.histplot(data=train, x='Group', hue='Transported', binwidth=1)
plt.title('Group')

plt.subplot(1,2,2)
sns.countplot(data=train, x='Group_size', hue='Transported')
plt.title('Group size')
fig.tight_layout()

그룹 사이즈를 1(less transported), 2~6(more transpported), 7이상 (equal)

In [ ]:
# 새 피쳐 train set
train['SoloGroup']=(train['Group_size']==1).astype(int) #원핫인코딩
train['MediumGroup']=((train['Group_size']>1) & (train['Group_size']<7)).astype(int) 
train['LargeGroup']=(train['Group_size']>=1).astype(int) 


# 새 피쳐 test set
test['SoloGroup']=(test['Group_size']==1).astype(int) #원핫인코딩
test['MediumGroup']=((test['Group_size']>1) & (test['Group_size']<7)).astype(int) 
test['LargeGroup']=(test['Group_size']>=1).astype(int) 


# 그래프


plt.figure(figsize=(10,4))
train['passenger_group_plot']=(train['SoloGroup']+2*train['MediumGroup']+3*train['LargeGroup']).astype(int)
sns.countplot(data=train, x='passenger_group_plot', hue='Transported')
plt.title('Passenger group distribution')
train.drop('passenger_group_plot', axis=1, inplace=True)

Cabin 선실

In [ ]:
#  NaN을 아웃라이어로 바꿔놓기
train['Cabin'].fillna('Z/9999/Z', inplace=True)
test['Cabin'].fillna('Z/9999/Z', inplace=True)

# 트레인
train['Cabin_number'] = train['Cabin'].apply(lambda x: x.split('/')[1]).astype(int)

# 테스트
test['Cabin_number'] = test['Cabin'].apply(lambda x: x.split('/')[1]).astype(int)

# NAN값 다시 넣기
train.loc[train['Cabin_number']==9999, 'Cabin_number']=np.nan
test.loc[test['Cabin_number']==9999, 'Cabin_number']=np.nan



#그래프
fig=plt.figure(figsize=(10,12))
sns.histplot(data=train, x='Cabin_number', hue='Transported',binwidth=20)
plt.vlines(300, ymin=0, ymax=200, color='black')
plt.vlines(600, ymin=0, ymax=200, color='black')
plt.vlines(900, ymin=0, ymax=200, color='black')
plt.vlines(1200, ymin=0, ymax=200, color='black')
plt.vlines(1500, ymin=0, ymax=200, color='black')
plt.vlines(1800, ymin=0, ymax=200, color='black')
plt.title('Cabin number')
plt.xlim([0,2000])

In [ ]:
# 새피처 -원핫인코딩
train['Cabin_region1']=(train['Cabin_number']<300).astype(int)   
train['Cabin_region2']=((train['Cabin_number']>=300) & (train['Cabin_number']<600)).astype(int)
train['Cabin_region3']=((train['Cabin_number']>=600) & (train['Cabin_number']<1210)).astype(int)
train['Cabin_region4']=((train['Cabin_number']>=1210) & (train['Cabin_number']<1800)).astype(int)
train['Cabin_region5']=(train['Cabin_number']>=1800).astype(int)

#새피처 test
test['Cabin_region1']=(test['Cabin_number']<300).astype(int)   
test['Cabin_region2']=((test['Cabin_number']>=300) & (test['Cabin_number']<600)).astype(int)
test['Cabin_region3']=((test['Cabin_number']>=600) & (test['Cabin_number']<1210)).astype(int)
test['Cabin_region4']=((test['Cabin_number']>=1210) & (test['Cabin_number']<1800)).astype(int)
test['Cabin_region5']=(test['Cabin_number']>=1800).astype(int)

# 그래프
plt.figure(figsize=(10,4))
train['Cabin_regions_plot']=(train['Cabin_region1']+2*train['Cabin_region2']+3*train['Cabin_region3']+4*train['Cabin_region4']+5*train['Cabin_region5']).astype(int)
sns.countplot(data=train, x='Cabin_regions_plot', hue='Transported')
plt.title('Cabin regions')
train.drop('Cabin_regions_plot', axis=1, inplace=True)

이름 제거

In [ ]:
# Replace NaN's with outliers for now (so we can split feature)
train['Name'].fillna('Unknown Unknown', inplace=True)
test['Name'].fillna('Unknown Unknown', inplace=True)

# Drop name (we don't need it anymore)
train.drop('Name', axis=1, inplace=True)
test.drop('Name', axis=1, inplace=True)

# 결측치 (Missing values)

In [ ]:
train.isna().sum()

In [ ]:

HPD_gb=train.groupby(['HomePlanet','Destination'])['Destination'].size().unstack().fillna(0)

#히트맵
plt.figure(figsize=(10,4))
sns.heatmap(HPD_gb.T, annot=True, fmt='g', cmap='coolwarm')

In [ ]:
train.loc[(train['HomePlanet'].isna()) & (~(train['Destination']=='55 Canccri e')), 'HomePlanet']='Earth'
train.loc[(train['HomePlanet'].isna()) & (train['Destination']=='55 Canccri e'), 'HomePlanet']='Europa'

In [ ]:
test.loc[(test['HomePlanet'].isna()) & (~(test['Destination']=='55 Canccri e')), 'HomePlanet']='Earth'
test.loc[(test['HomePlanet'].isna()) & (test['Destination']=='55 Canccri e'), 'HomePlanet']='Europa'

In [ ]:
train.isna().sum()

In [ ]:
test.isna().sum()

In [ ]:
train.groupby(['HomePlanet','SoloGroup','Cabin_number'])['Age'].median().unstack()

In [ ]:
na_rows_A=train.loc[train['Age'].isna(),'Age'].index


train.loc[train['Age'].isna(),'Age']=train.groupby(['SoloGroup'])['Age'].transform(lambda x: x.fillna(x.median()))[na_rows_A]

In [ ]:
na_rows_B=test.loc[test['Age'].isna(),'Age'].index


test.loc[test['Age'].isna(),'Age']=test.groupby(['SoloGroup'])['Age'].transform(lambda x: x.fillna(x.median()))[na_rows_B]

In [ ]:
# update
train.loc[train['Age']<=5,'Age_group']='Age_0-5'
train.loc[(train['Age']>5) & (train['Age']<=18),'Age_group']='Age_6-18'
train.loc[(train['Age']>18) & (train['Age']<=25),'Age_group']='Age_19-25'
train.loc[(train['Age']>25) & (train['Age']<=40),'Age_group']='Age_26-40'
train.loc[train['Age']>40,'Age_group']='Age_41+'


test.loc[test['Age']<=5,'Age_group']='Age_0-5'
test.loc[(test['Age']>5) & (test['Age']<=18),'Age_group']='Age_6-18'
test.loc[(test['Age']>18) & (test['Age']<=25),'Age_group']='Age_19-25'
test.loc[(test['Age']>25) & (test['Age']<=40),'Age_group']='Age_26-40'
test.loc[test['Age']>40,'Age_group']='Age_41+'

In [ ]:
train.groupby(['SoloGroup','CryoSleep'])['CryoSleep'].size().unstack().fillna(0)

In [ ]:
na_rows_CSL=train.loc[train['CryoSleep'].isna(),'CryoSleep'].index
train.loc[train['CryoSleep'].isna(),'CryoSleep']=train.groupby(['SoloGroup'])['CryoSleep'].transform(lambda x: x.fillna(pd.Series.mode(x)[0]))[na_rows_CSL]

In [ ]:
na_rows_CSL=test.loc[test['CryoSleep'].isna(),'CryoSleep'].index
test.loc[test['CryoSleep'].isna(),'CryoSleep']=test.groupby(['SoloGroup'])['CryoSleep'].transform(lambda x: x.fillna(pd.Series.mode(x)[0]))[na_rows_CSL]

In [ ]:
test.isna().sum()

# 전처리

In [ ]:
#X=train.copy(deep=True)
#X_test=test.copy(deep=True)

In [ ]:
#X.head()

In [ ]:
# 필요없는 것들 삭제 
train.drop(["PassengerId", "Group", "Group_size", "Age_group", "Cabin","Cabin_number","Destination","HomePlanet"], axis=1, inplace=True)
test.drop(["PassengerId", "Group", "Group_size", "Age_group", "Cabin","Cabin_number","Destination","HomePlanet"], axis=1, inplace=True)

In [ ]:
#X.dtypes

# 모델

In [ ]:
X = train.drop(["Transported"], axis=1)
y = train["Transported"]

le = LabelEncoder()
y = le.fit_transform(y)
print(le.classes_)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

In [ ]:
X_train.isna().sum()

In [ ]:
lg = LogisticRegression()
lg.fit(X_train, y_train)
lg_pred = lg.predict(X_test)
print(round(accuracy_score(y_test, lg_pred), 3))

In [ ]:
dt = DecisionTreeClassifier()
dt.fit(X_train, y_train)
dt_pred = dt.predict(X_test)
print(round(accuracy_score(y_test, dt_pred), 3))

In [ ]:
rf = RandomForestClassifier()
rf.fit(X_train, y_train)
rf_pred = rf.predict(X_test)
print(round(accuracy_score(y_test, rf_pred), 3))

In [ ]:
#sub_df = pd.read_csv("/kaggle/input/spaceship-titanic/sample_submission.csv")
#sub_df

In [ ]:
final_pred = lg.predict(test)
final_pred = le.inverse_transform(final_pred)

sub_df["Transported"] = final_pred
sub_df.head()

In [ ]:
sub_df.to_csv("1815364.csv", index=False)